# Exercise 13

This particular Automobile Data Set includes a good mix of categorical values as well as continuous values and serves as a useful example that is relatively easy to understand. Since domain understanding is an important aspect when deciding how to encode various categorical values - this data set makes a good case study.

Read the data into Pandas

In [1]:
import pandas as pd

# Define the headers since the data does not have any
headers = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration",
           "num_doors", "body_style", "drive_wheels", "engine_location",
           "wheel_base", "length", "width", "height", "curb_weight",
           "engine_type", "num_cylinders", "engine_size", "fuel_system",
           "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm",
           "city_mpg", "highway_mpg", "price"]

# Read in the CSV file and convert "?" to NaN
df = pd.read_csv("http://mlr.cs.umass.edu/ml/machine-learning-databases/autos/imports-85.data",
                  header=None, names=headers, na_values="?" )
df.head()

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheel_base,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
2,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
3,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
4,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0


In [2]:
df.shape

(205, 26)

In [3]:
df.dtypes

symboling              int64
normalized_losses    float64
make                  object
fuel_type             object
aspiration            object
num_doors             object
body_style            object
drive_wheels          object
engine_location       object
wheel_base           float64
length               float64
width                float64
height               float64
curb_weight            int64
engine_type           object
num_cylinders         object
engine_size            int64
fuel_system           object
bore                 float64
stroke               float64
compression_ratio    float64
horsepower           float64
peak_rpm             float64
city_mpg               int64
highway_mpg            int64
price                float64
dtype: object

In [4]:
obj_df = df.select_dtypes(include=['object']).copy()
obj_df.columns

Index(['make', 'fuel_type', 'aspiration', 'num_doors', 'body_style',
       'drive_wheels', 'engine_location', 'engine_type', 'num_cylinders',
       'fuel_system'],
      dtype='object')

# Exercise 13.1

Does the database contain missing values? If so, replace them using one of the methods explained in class

In [5]:
#Looking for missing values
missing_values = df.loc[:, (df.isnull().sum()>0)]
missing_values.dtypes

normalized_losses    float64
num_doors             object
bore                 float64
stroke               float64
horsepower           float64
peak_rpm             float64
price                float64
dtype: object

In [6]:
#for continuous values, replace missing values with median 
cont_values = ['normalized_losses', 'bore', 'stroke', 'horsepower','peak_rpm', 'price']

for i in cont_values:
    df[i].fillna(df[i].median(), inplace=True)

#for categorical values, use backward fill
df['num_doors'].fillna(method='bfill', inplace=True)

In [7]:
df.isnull().sum()

symboling            0
normalized_losses    0
make                 0
fuel_type            0
aspiration           0
num_doors            0
body_style           0
drive_wheels         0
engine_location      0
wheel_base           0
length               0
width                0
height               0
curb_weight          0
engine_type          0
num_cylinders        0
engine_size          0
fuel_system          0
bore                 0
stroke               0
compression_ratio    0
horsepower           0
peak_rpm             0
city_mpg             0
highway_mpg          0
price                0
dtype: int64

# Exercise 13.2

Split the data into training and testing sets

Train a Random Forest Regressor to predict the price of a car using the scalar features

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

In [9]:
#create X and y
X = df.select_dtypes(include=['int64', 'float64']).drop('price', axis=1).values
y = df['price'].values

#split data in train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=22)

#train the model
rfmodel = RandomForestRegressor(n_estimators=50, n_jobs=-1, random_state=22)
rfmodel.fit(X_train, y_train)
y_pred = rfmodel.predict(X_test)
scalar = np.sqrt(mean_squared_error(y_test, y_pred))
print(scalar)

2573.711383754929


# Exercise 13.3

Create dummy variables for the categorical features

Train a Random Forest Regressor and compare

In [10]:
#look for columns to drop
obj_df.columns

Index(['make', 'fuel_type', 'aspiration', 'num_doors', 'body_style',
       'drive_wheels', 'engine_location', 'engine_type', 'num_cylinders',
       'fuel_system'],
      dtype='object')

In [11]:
to_drop = ['make', 'fuel_type', 'aspiration', 'num_doors', 'body_style',
       'drive_wheels', 'engine_location', 'engine_type', 'num_cylinders',
       'fuel_system']

In [12]:
#create dummy variables
obj_df1 = pd.get_dummies(obj_df, prefix='D')

#merge both df
df_dummy = df.merge(obj_df1, left_index=True, right_index=True)

#drop unnecessary variables
df_dummy.drop(to_drop, axis=1, inplace=True)
df_dummy.head()

,symboling,normalized_losses,wheel_base,length,width,height,curb_weight,engine_size,bore,stroke,...,D_twelve,D_two,D_1bbl,D_2bbl,D_4bbl,D_idi,D_mfi,D_mpfi,D_spdi,D_spfi
0,3,115.0,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,...,0,0,0,0,0,0,0,1,0,0
1,3,115.0,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,...,0,0,0,0,0,0,0,1,0,0
2,1,115.0,94.5,171.2,65.5,52.4,2823,152,2.68,3.47,...,0,0,0,0,0,0,0,1,0,0
3,2,164.0,99.8,176.6,66.2,54.3,2337,109,3.19,3.40,...,0,0,0,0,0,0,0,1,0,0
4,2,164.0,99.4,176.6,66.4,54.3,2824,136,3.19,3.40,...,0,0,0,0,0,0,0,1,0,0


In [13]:
#create X and y
X = df_dummy.drop('price', axis=1).values
y = df_dummy['price'].values

#split data in train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=22)

#train the model
rfmodel = RandomForestRegressor(n_estimators=50, n_jobs=-1, random_state=22)
rfmodel.fit(X_train, y_train)
y_pred = rfmodel.predict(X_test)
dummy = np.sqrt(mean_squared_error(y_test, y_pred))
print(dummy)

2418.9834130659456


With no tunning of the model, the result gets better when adding dummy variables as features.

# Exercise 13.4

Apply two other methods of categorical encoding

compare the results

In [14]:
import category_encoders as ce

#### Using Binary Encode

In [15]:
#Binary encode the categorical variables
obj_df2 = ce.BinaryEncoder().fit_transform(obj_df)

#merge both df
df_be = df.merge(obj_df2, left_index=True, right_index=True)

#drop unnecessary variables
df_be.drop(to_drop, axis=1, inplace=True)
df_be.head()

,symboling,normalized_losses,wheel_base,length,width,height,curb_weight,engine_size,bore,stroke,...,engine_type_2,engine_type_3,num_cylinders_0,num_cylinders_1,num_cylinders_2,num_cylinders_3,fuel_system_0,fuel_system_1,fuel_system_2,fuel_system_3
0,3,115.0,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,...,0,1,0,0,0,1,0,0,0,1
1,3,115.0,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,...,0,1,0,0,0,1,0,0,0,1
2,1,115.0,94.5,171.2,65.5,52.4,2823,152,2.68,3.47,...,1,0,0,0,1,0,0,0,0,1
3,2,164.0,99.8,176.6,66.2,54.3,2337,109,3.19,3.40,...,1,1,0,0,0,1,0,0,0,1
4,2,164.0,99.4,176.6,66.4,54.3,2824,136,3.19,3.40,...,1,1,0,0,1,1,0,0,0,1


In [16]:
#create X and y
X = df_be.drop('price', axis=1).values
y = df_be['price'].values

#split data in train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=22)

#train the model
rfmodel = RandomForestRegressor(n_estimators=50, n_jobs=-1, random_state=22)
rfmodel.fit(X_train, y_train)
y_pred = rfmodel.predict(X_test)
binary_encode = np.sqrt(mean_squared_error(y_test, y_pred))
print(binary_encode)

2517.1558754199023


#### Ordinal

In [17]:
#Ordinal encode the categorical variables
obj_df3 = ce.OrdinalEncoder().fit_transform(obj_df)

#merge both df
df_ord = df.merge(obj_df3, left_index=True, right_index=True, suffixes=('_x', '_o'))

#drop unnecessary variables
for i in range(len(to_drop)):
    to_drop[i] = to_drop[i]+'_x'

df_ord.drop(to_drop, axis=1, inplace=True)
df_ord.head()

,symboling,normalized_losses,wheel_base,length,width,height,curb_weight,engine_size,bore,stroke,...,make_o,fuel_type_o,aspiration_o,num_doors_o,body_style_o,drive_wheels_o,engine_location_o,engine_type_o,num_cylinders_o,fuel_system_o
0,3,115.0,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,...,1,1,1,1,1,1,1,1,1,1
1,3,115.0,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,...,1,1,1,1,1,1,1,1,1,1
2,1,115.0,94.5,171.2,65.5,52.4,2823,152,2.68,3.47,...,1,1,1,1,2,1,1,2,2,1
3,2,164.0,99.8,176.6,66.2,54.3,2337,109,3.19,3.40,...,2,1,1,2,3,2,1,3,1,1
4,2,164.0,99.4,176.6,66.4,54.3,2824,136,3.19,3.40,...,2,1,1,2,3,3,1,3,3,1


In [18]:
#create X and y
X = df_ord.drop('price', axis=1).values
y = df_ord['price'].values

#split data in train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=22)

#train the model
rfmodel = RandomForestRegressor(n_estimators=50, n_jobs=-1, random_state=22)
rfmodel.fit(X_train, y_train)
y_pred = rfmodel.predict(X_test)
ordinal = np.sqrt(mean_squared_error(y_test, y_pred))
print(ordinal)

2590.329964873595


In [19]:
results = pd.DataFrame({'RSME':[scalar, dummy, binary_encode, ordinal]}, index=['Scalar','Dummies','Binary Encoder','Ordinal'])
results.sort_values('RSME')

,RSME
Dummies,2418.983413
Binary Encoder,2517.155875
Scalar,2573.711384
Ordinal,2590.329965
